<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Sentiment Analysis and Naive Bayes

_Instructor: Aymeric Flaisler_

---

In the sentiment analysis lesson we used a predefined dictionary of positive and negative valences for words. This  lab has invert the process: you'll find which words are most likely to appear in positive or negative reviews by using the rotten vs. fresh binary label.

### Naive Bayes

A practical and common way to do this is with the Naive Bayes algorithm. For this lab you'll  be leveraging the sklearn implementation.

Given a feature $x_i$ and target $y_i$, Naive Bayes classifiers solve for $P(x_i \;|\; y_i)$. In other words, the probability of a feature/predictor _given_ that the target is 1.

We'll use this to figure out which words are more likely to appear when the target is 1 ("fresh") vs when the target is 0 ("rotten").

---

### 1. Load packages and movie data

Do any cleaning you deem necessary.

In [27]:
import pandas as pd
import numpy as np

# We are using the BernoulliNB version of Naive Bayes, which assumes predictors are binary encoded.
from sklearn.naive_bayes import BernoulliNB
from sklearn.model_selection import cross_val_score, train_test_split

from sklearn.feature_extraction.text import CountVectorizer


In [2]:
rt = pd.read_csv('../datasets/rt_critics.csv')

rt = rt[rt.fresh.isin(['fresh','rotten'])]
rt.fresh = rt.fresh.map(lambda x: 1 if x == 'fresh' else 0)

In [3]:
rt.head(2)

,critic,fresh,imdb,publication,quote,review_date,rtid,title
0,Derek Adams,1,114709.0,Time Out,"So ingenious in concept, design and execution ...",2009-10-04,9559.0,Toy story
1,Richard Corliss,1,114709.0,TIME Magazine,The year's most inventive comedy.,2008-08-31,9559.0,Toy story


---

### 2. Create a predictor matrix of words from the quotes with CountVectorizer

It is up to you what ngram range you want to select. **Make sure that `binary=True`**

In [9]:
cv = CountVectorizer(ngram_range=(1,2), max_features=2500, binary=True, stop_words='english')
words = cv.fit_transform(rt.quote)

In [10]:
words.shape

(14049, 2500)

In [11]:
words = pd.DataFrame(words.todense(), columns=cv.get_feature_names())

In [12]:
words.head()

10  100  13  1961  1998  20  2001  30  40  50s  70s  80s  90  90s  ability  \
0   0    0   0     0     0   0     0   0   0    0    0    0   0    0        0   
1   0    0   0     0     0   0     0   0   0    0    0    0   0    0        0   
2   0    0   0     0     0   0     0   0   0    0    0    0   0    0        0   
3   0    0   0     0     0   0     0   0   0    0    0    0   0    0        0   
4   0    0   0     0     0   0     0   0   0    0    0    0   0    0        0   

   able  absence  absolutely  absorbing  absurd  accomplished  account  \
0     0        0           0          0       0             0        0   
1     0        0           0          0       0             0        0   
2     0        0           0          0       0             0        0   
3     0        0           0          0       0             0        0   
4     0        0           0          0       0             0        0   

   achieve  achieved  achievement  achievements  achieves  act  acted  acting  \
0        0         0            0             0         0    0      0       0   
1        0         0            0             0         0    0      0       0   
2        0         0            0             0         0    0      0       0   
3        0         0            1             0         0    0      0       0   
4        0         0            0             0         0    0      0       0   

   action  action adventure  action movie  action scenes  action sequences  \
0       0                 0             0              0                 0   
1       0                 0             0              0                 0   
2       0                 0             0              0                 0   
3       0                 0             0              0                 0   
4       0                 0             0              0                 0   

   actor  actors  actress  actual  actually  adam  adaptation  adapted  add  \
0      0       0        0       0         0     0           0        0    0   
1      0       0        0       0         0     0           0        0    0   
2      0       0        0       0         0     0           0        0    0   
3      0       0        0       0         0     0           0        0    0   
4      0       0        0       0         0     0           0        0    0   

   added  addition  adds  admirable  admire  admit  adolescent  adult  adults  \
0      0         0     0          0       0      0           0      0       0   
1      0         0     0          0       0      0           0      0       0   
2      0         0     0          0       0      0           0      0       0   
3      0         0     0          0       0      0           0      0       0   
4      0         0     0          0       0      0           0      0       0   

   advantage  adventure  adventures  aesthetic  affair  affecting  affection  \
0          0          0           0          0       0          0          0   
1          0          0           0          0       0          0          0   
2          0          0           0          0       0          0          0   
3          0          0           0          0       0          0          0   
4          0          0           0          0       0          0          0   

   affectionate  affleck  africa  age  ages  ago  agreeable  ahead  aimed  \
0             0        0       0    0     0    0          0      0      0   
1             0        0       0    0     0    0          0      0      0   
2             0        0       0    1     0    0          0      0      0   
3             0        0       0    0     0    0          0      0      0   
4             0        0       0    0     0    0          0      0      0   

   aims  air  alfred  alfred hitchcock  alien  alive  allen  allowed  allows  \
0     0    0       0                 0      0      0      0        0       0   
1     0    0       0                 0      0    

In [8]:
print(words.shape)

(14049, 2500)


---

### 3. Split data into training and testing splits

You should keep 25% of the data in the test set.

In [15]:
Xtrain, Xtest, ytrain, ytest = train_test_split(words.values, rt.fresh.values, test_size=0.25)

In [16]:
print(Xtrain.shape, Xtest.shape)

(10536, 2500) (3513, 2500)


---

### 4. Build a `BernoulliNB` model predicting fresh vs. rotten from the word appearances

The model should only be built (and cross-validated) on the training data.

Cross-validate the score and compare it to baseline.

In [17]:
# because we are using the binary feature here it is best to use 
# BernoulliNB
nb = BernoulliNB()

In [18]:
nb.fit(Xtrain, ytrain)

BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True)

In [19]:
nb_scores = cross_val_score(BernoulliNB(), Xtrain, ytrain, cv=5)
print(nb_scores)
print(np.mean(nb_scores))
print(np.mean(ytrain))

[0.7300759  0.73055028 0.73706692 0.73137162 0.74074074]
0.733961092980995
0.6082953682611997


---

### 5. Pull out the probability of words given "fresh"

The `.feature_log_prob_` attribute of the naive bayes model contains the log probabilities of a feature appearing given a target class.

The rows correspond to the class of the target, and the columns correspond to the features. The first row is the 0 "rotten" class, and the second is the 1 "fresh" class.

#### 5.1 Pull out the log probabilities and convert them to probabilities (for fresh and for rotten).

In [26]:
feat_lp = nb.feature_log_prob_
feat_lp.shape

(2, 2500)

In [24]:
fresh_p = np.exp(feat_lp[1])
fresh_p[:5]

array([0.00218375, 0.00109187, 0.00109187, 0.00155982, 0.00093589])

In [25]:
rotten_p = np.exp(feat_lp[0])
rotten_p[:5]

array([0.00387503, 0.00121095, 0.00072657, 0.00024219, 0.00121095])

#### 5.2 Make a dataframe with the probabilities and features

In [ ]:
feat_probs = pd.DataFrame({'fresh_p':fresh_p, 'rotten_p':rotten_p, 'feature':words.columns.values})

#### 5.3 Create a column that is the difference between fresh probability of appearance and rotten

In [ ]:
feat_probs['fresh_diff'] = feat_probs.fresh_p - feat_probs.rotten_p

#### 5.4 Look at the most likely words for fresh and rotten reviews

In [ ]:
feat_probs.sort_values('fresh_diff', ascending=False, inplace=True)
feat_probs.head(20)

In [ ]:
feat_probs.sort_values('fresh_diff', ascending=True, inplace=True)
feat_probs.head(20)

---

### 6. Examine how your model performs on the test set

In [ ]:
print(nb.score(Xtest, ytest))
print(np.mean(ytest))

---

### 7. Look at the top 10 movies and reviews likely to be fresh and top 10 likely to be rotten

You can fit the model on the full set of data for this.

> **Note:** Naive Bayes, while good at classifying, is known to be somewhat bad at giving accurate predicted probabilities (beyond getting it on the correct side of 50%). It is a good classifier but a bad estimator. 

In [ ]:
X = words.values
y = rt.fresh

In [ ]:
nbfull = BernoulliNB().fit(X,y)

In [ ]:
pp = pd.DataFrame({
        'prob_fresh':nbfull.predict_proba(X)[:,1],
        'movie':rt.title,
        'quote':rt.quote
    })

In [ ]:
pp.sort_values('prob_fresh', ascending=False, inplace=True)
for movie, quote in zip(pp.movie[0:10], pp.quote[0:10]):
    print(movie,'\t', quote)
    print('--------------------------------------------------\n')

In [ ]:
pp.sort_values('prob_fresh', ascending=True, inplace=True)
for movie, quote in zip(pp.movie[0:10], pp.quote[0:10]):
    print(movie,'\t', quote)
    print('--------------------------------------------------\n')

---

### 8. Find the most likely to be fresh and rotten for movies with at least 10 reviews.

In [ ]:
# subset to movies with at least 10 reviews:
movie_counts = pp.movie.value_counts().reset_index()
movie_counts.columns = ['movie','counts']
movie_counts.head()

In [ ]:
pp_movies = pp[['movie','prob_fresh']].groupby('movie').agg(np.mean).reset_index()
pp_movies = pp_movies[pp_movies.movie.isin(movie_counts[movie_counts.counts >= 10].movie)]

In [ ]:
pp_movies.sort_values('prob_fresh', ascending=False, inplace=True)
pp_movies.head(20)

In [ ]:
pp_movies.sort_values('prob_fresh', ascending=True, inplace=True)
pp_movies.head(20)